# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1 Подготовка

In [1]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.dummy import DummyRegressor
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

Загрузим наши данные.

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv') 
df.head(20)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Далее мы оставляем только 400 строк, сокращая время эмбендинга.

In [3]:
data = df.sample(1000).reset_index(drop=True) 

Очистим текст от всего лишнего перед токенизацией.

In [4]:
def clear_text(text):
    i = re.sub(r'[^a-zA-Z,]', ' ', text)
    i = i.split()
    return " ".join(i)

In [5]:
data['clear_text'] = data['text'].apply(clear_text)

In [6]:
data.head(20)

,Unnamed: 0,text,toxic,clear_text
0,58300,Georgi Kinkladze \n\nI took the article to FAC...,0,Georgi Kinkladze I took the article to FAC twi...
1,41365,January 2014 (UTC)\n\nJust a further reply to ...,0,January UTC Just a further reply to the editor...
2,79460,"3RR warning\nNice try, but I read and it was s...",0,"RR warning Nice try, but I read and it was sou..."
3,85014,"""\n\nI know you posted a source but unfortunat...",0,I know you posted a source but unfortunately i...
4,152968,Wikipedia:Requests for arbitration/Free Republ...,0,Wikipedia Requests for arbitration Free Republ...
5,150045,"""\nI propose to amend the biography section al...",0,I propose to amend the biography section along...
6,143486,. It picks up on weasel words when they are pr...,0,It picks up on weasel words when they are pres...
7,79269,not done\n\nIt is seriously not done to remove...,0,not done It is seriously not done to remove th...
8,12374,"If people revert my edits for no good reason, ...",1,"If people revert my edits for no good reason, ..."
9,10227,Cite a reliable source for your edits.,0,Cite a reliable source for your edits


In [7]:
# инициализируем токенизатор
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
features = data['clear_text']

In [8]:
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True)
#features = data['clear_text']

In [9]:
# создадим функцию, которая токенизирует каждый твит,
# находит максимальную длину векторов после токенизации
# применяет к векторам padding
# создает маску для выделения важных токенов

def tokenize(features, max_len=0):
    tokenized = features.apply(lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True))
        
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)
    return padded, attention_mask, max_len

In [10]:
padded, attention_mask, max_len = tokenize(features, 0)

In [11]:
padded.shape

(1000, 512)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# затем инициализируем саму модель класса BertModel. Передадим ей файл с предобученной моделью и конфигурацией: 
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert", output_hidden_states=True).to(device)

In [14]:
# Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим:
batch_size = 50

# Сделаем цикл по батчам. Отображать прогресс будет функция notebook():
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):

        # Преобразуем данные в формат тензоров (англ. tensor) — многомерных векторов в библиотеке torch.
        #Тип данных LongTensor (англ. «длинный тензор») хранит числа в «длинном формате»,
        #то есть выделяет на каждое число 64 бита.

    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad(): 
        #Для ускорения вычисления пользуемся функцией no_grad() 
        #Чтобы получить эмбеддинги для батча, передадим модели данные и маску:
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))
        
        # Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов:
        embeddings.append(batch_embeddings.hidden_states[-1][:, 0, :].cpu().numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

## Обучение модели

In [15]:
# Соберём все эмбеддинги в матрицу признаков вызовом функции concatenate():
features = np.concatenate(embeddings)


target = data['toxic']
x_train, x_test, y_train, y_test = train_test_split(features, target,  test_size = 0.2, random_state=12345)

# проверим размеры полученных выборок
print('Размеры обучающей выборки:')
print(x_train.shape)
print(y_train.shape)
print()
print('Размеры тестовой выборки:')
print(x_test.shape)
print(y_test.shape) 

Размеры обучающей выборки:
(800, 768)
(800,)

Размеры тестовой выборки:
(200, 768)
(200,)


In [16]:
# создадим функцию, которая будет подбирать гиперпараметры для указанной ей модели
# затем данная функция будет проводить кросс-валидацию по метрике F1_score

def score(model_name, params, features, target):
    model = model_name
    parametrs = params    
    grid = GridSearchCV(model, parametrs, scoring='f1', cv=3) 
    grid.fit(features, target)    
    print('f1_score модели:', grid.best_score_)
    print('лучшие параметры:', grid.best_estimator_)
    return

In [17]:
score(LogisticRegression(random_state=12345), {'C':[1, 10, 100]}, 
                                                x_train, y_train)

C:\Users\123\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\123\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = 

f1_score модели: 0.9177553813612045
лучшие параметры: LogisticRegression(C=100, random_state=12345)


Модель показала следующие результаты: f1_score модели: 0.917, лучшие параметры: LogisticRegression(C=100,random_state=12345)

In [18]:
score(DecisionTreeClassifier(random_state=12345),{'max_depth': range (1, 13, 2),
                                                   'min_samples_leaf': range (1, 11)}, 
                                                    x_train, y_train) 

f1_score модели: 0.8744398875977822
лучшие параметры: DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, random_state=12345)


Модель показала следующие результаты: f1_score модели: 0.874,
лучшие параметры: DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, random_state=12345)

In [19]:
score(RandomForestClassifier(random_state=12345),{'max_depth': range (5, 10, 1),                          
                                                  'n_estimators': range (50, 100, 10)}, 
                                                   x_train, y_train)

f1_score модели: 0.9319727891156463
лучшие параметры: RandomForestClassifier(max_depth=5, n_estimators=50, random_state=12345)


Модель показала следующие результаты: f1_score модели: 0.904, лучшие параметры: RandomForestClassifier(max_depth=5, n_estimators=60, random_state=12345)

## Проверка модели

**Подводя итоги, можно сделать следующие выводы:**

1. LogisticRegression(C=100,random_state=12345) - Результат подходит под требуемый: f1_score модели:  0.917.


2. DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, random_state=12345) - Результат подходит под требуемый: f1_score модели: 0.874


3. RandomForestClassifier(max_depth=5, n_estimators=50, random_state=12345) - Результат подходит под требуемый: f1_score модели: 0.931


Лучше всего себя показала модель `RandomForestClassifier`, она имеет лучший показатель метрики F1.

In [20]:
# на основе вышесказанного проверим модель RandomForestClassifier на тестовой выборке


model = RandomForestClassifier(max_depth=5, n_estimators=50, random_state=12345)
model.fit(x_train, y_train)

pred = model.predict(x_test)
f1_score(y_test, pred)

0.8823529411764706

In [21]:
# создаём модель DummyRegressor,
# которая предсказывает все значения
# как медиану массива x_test 

#dummy_regr = DummyRegressor(strategy='median')
#dummy_regr.fit(x_train, y_train)

#predict_for_check = dummy_regr.predict(x_test)
#f1_score(y_test, predict_for_check)


# Здесь была попытка создать константную модель, но у нее как ни крути F1 = 0, слишком нелепо выглядит 